In [1]:
from brayns import Client
from circuitexplorer import CircuitExplorer
brayns = Client('localhost:5000')
ce = CircuitExplorer(brayns)
config = '/media/favreau/Documents/medias/astrocytes/one.astrocytes'

In [2]:
model = ce.load_astrocytes(path=config, use_sdf=True, generate_internals=True)

In [3]:
brayns.set_camera(
 orientation = [0.0, 0.7071067811865475, 0.0, 0.7071067811865476],
 position = [534.6766202822544, 696.5267742832598, 921.705842287376],
 target = [423.8168899214437, 696.5267742832598, 921.705842287376],
)

True

In [4]:
def set_materials(model_id, palette_name, opacity=1.0):
    
    opacities=list()
    refraction_indices=list()
    reflection_indices=list()
    shading_modes=list()
    diffuse_colors=list()
    specular_colors=list()
    specular_exponents=list()
    material_ids=list()
    glossinesses=list()
    user_parameters=list()
    
    import seaborn as sns
    material_ids = ce.get_material_ids(model_id)['ids']
    nb_materials = len(material_ids)
    palette = sns.color_palette(palette_name, nb_materials)
    
    for i in range(nb_materials):
        c=palette[i]
        mid = material_ids[i] % 10
        if mid == ce.MATERIAL_OFFSET_MITOCHONDRION:
            opacities.append(1.0)
            glossinesses.append(0.7)
            shading_modes.append(ce.SHADING_MODE_PERLIN)
            user_parameters.append(0.08)
            specular_exponents.append(5)
        else:
            opacities.append(opacity)
            glossinesses.append(1.0)
            shading_modes.append(ce.SHADING_MODE_ELECTRON)
            user_parameters.append(1.0)
            specular_exponents.append(50)

        refraction_indices.append(1.0)
        reflection_indices.append(0.0)
        diffuse_colors.append([c[0],c[1],c[2]])
        specular_colors.append([c[0],c[1],c[2]])
        

    ce.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        opacities=opacities, reflection_indices=reflection_indices,
        shading_modes=shading_modes, user_parameters=user_parameters,
        diffuse_colors=diffuse_colors, specular_colors=specular_colors,
        specular_exponents=specular_exponents, glossinesses=glossinesses,
        refraction_indices=refraction_indices)

In [5]:
model_id = model['id']
ce.set_material_extra_attributes(model_id)
set_materials(model_id, 'Set1', 0.5)

In [6]:
status = brayns.set_renderer(
    subsampling=4, max_accum_frames=16)
params = brayns.CircuitExplorerAdvancedRendererParams()
params.max_bounces = 10
params.epsilon_factor = 10
status = brayns.set_renderer_params(params)